# Welcome to the OGD.AT Lab!

<a href="https://data.gv.at"><img align="right" src="./img/data-gv-at.png"></a>

This lab provides **Jupyter notebooks** for working with **Austrian open government data** and other auxilliary open data sources. 


## Lab notebooks

1. [Accessing geodata from data.wien.gv.at services](wien-ogd.ipynb)
1. [Geocoding addresses](geocoding.ipynb)